# Handle missing data for movie metadata

In [20]:
import pandas as pd

df = pd.read_csv('../../datasets/input/movies_metadata.csv',
                 dtype={'id': str},
                 low_memory=False)
data = df.copy()

# Movies metadata analysis

In [21]:
# Check for missing data in each column
print("Columns with missing data:\n")

missing_data = []
for column in data.columns:
    missing_count = data[column].isnull().sum()
    if missing_count > 0:
        missing_pct = (missing_count / len(data)) * 100
        missing_data.append({
            'Column': column,
            'Missing Count': missing_count,
            'Missing %': f"{missing_pct:.2f}%"
        })
        print(f"{column:30} | {missing_count:6} missing ({missing_pct:5.2f}%)")

print(f"Total columns: {len(data.columns)}")
print(f"Columns with missing data: {len(missing_data)}")
print(f"Columns without missing data: {len(data.columns) - len(missing_data)}")

Columns with missing data:

belongs_to_collection          |  40972 missing (90.12%)
homepage                       |  37684 missing (82.88%)
imdb_id                        |     17 missing ( 0.04%)
original_language              |     11 missing ( 0.02%)
overview                       |    954 missing ( 2.10%)
popularity                     |      5 missing ( 0.01%)
poster_path                    |    386 missing ( 0.85%)
production_companies           |      3 missing ( 0.01%)
production_countries           |      3 missing ( 0.01%)
release_date                   |     87 missing ( 0.19%)
revenue                        |      6 missing ( 0.01%)
runtime                        |    263 missing ( 0.58%)
spoken_languages               |      6 missing ( 0.01%)
status                         |     87 missing ( 0.19%)
tagline                        |  25054 missing (55.10%)
title                          |      6 missing ( 0.01%)
video                          |      6 missing ( 0.01%)
vot

# Step 1: Fix malformed id field

In [22]:
# Convert id to numeric, coercing errors to NaN
print(f"Original row count: {len(data)}")

data['id'] = pd.to_numeric(data['id'], errors='coerce')

# Count how many rows have malformed id
malformed_id_count = data['id'].isnull().sum()

# Show examples of malformed rows (before dropping)
if malformed_id_count > 0:
    print(data[data['id'].isnull()][['id', 'title', 'original_title']].head())

Original row count: 45466
       id title                            original_title
19730 NaN   NaN  [{'iso_639_1': 'en', 'name': 'English'}]
29503 NaN   NaN      [{'iso_639_1': 'ja', 'name': '日本語'}]
35587 NaN   NaN  [{'iso_639_1': 'en', 'name': 'English'}]


# Step 2: Drop rows with missing critical fields (id, title)

In [23]:
# Drop rows with missing id or title
rows_before = len(data)

data = data.dropna(subset=['id', 'title'])

rows_dropped = rows_before - len(data)
print(f"Rows dropped due to missing id or title: {rows_dropped}")
print(f"Remaining rows: {len(data)}")

# Convert id to integer
data['id'] = data['id'].astype(int)
print(f"\nData types after cleaning:")
print(data[['id', 'title']].dtypes)

Rows dropped due to missing id or title: 6
Remaining rows: 45460

Data types after cleaning:
id        int64
title    object
dtype: object


# Step 3: Parse JSON columns (genres, production_companies, etc.)

In [24]:
import ast

def parse_json_column(json_str):
    try:
        if pd.isna(json_str):
            return []
        data = ast.literal_eval(json_str)
        if isinstance(data, list):
            return [item['name'] for item in data if 'name' in item]
        return []
    except:
        return []

# Parse genres
data['genres_list'] = data['genres'].apply(parse_json_column)

# Parse production_companies
data['production_companies_list'] = data['production_companies'].apply(parse_json_column)

# Parse production_countries
data['production_countries_list'] = data['production_countries'].apply(parse_json_column)

# Parse spoken_languages
data['spoken_languages_list'] = data['spoken_languages'].apply(parse_json_column)

print(data[['title', 'genres_list', 'production_companies_list']].head(3))

              title                   genres_list  \
0         Toy Story   [Animation, Comedy, Family]   
1           Jumanji  [Adventure, Fantasy, Family]   
2  Grumpier Old Men             [Romance, Comedy]   

                           production_companies_list  
0                          [Pixar Animation Studios]  
1  [TriStar Pictures, Teitler Film, Interscope Co...  
2                     [Warner Bros., Lancaster Gate]  


# Step 4: Save cleaned dataset

In [25]:

columns_to_keep = [
    'id', 'title', 'original_title', 'release_date', 'runtime',
    'budget', 'revenue', 'popularity', 'vote_average', 'vote_count',
    'overview', 'tagline', 'imdb_id', 'original_language', 'status',
    'genres_list', 'production_companies_list', 
    'production_countries_list', 'spoken_languages_list'
]

cleaned_data = data[columns_to_keep].copy()

# Save to CSV
output_path = '../../datasets/output/cleaned_datasets/cleaned_movies_metadata.csv'
cleaned_data.to_csv(output_path, index=False)
